In [ ]:
import exp_voroni
import importlib
importlib.reload(exp_voroni)

In [ ]:
from exp_voroni import *
# -- plot traces voroni tiles
# one user
# sphere_plot_voro14_one_video_one_user()
# all user
# sphere_plot_voro14_one_video_all_users()

# -- plot traces rectan tiles
# erp_tiles_heatmap(get_traces_one_video_one_user())
# sphere_plot_rectan6x4_one_video_one_user()
sphere_plot_rectan4x4_one_video_one_user()


In [ ]:
# -- heatmap from_position_to_tile_center_covered
from exp_voroni import *

# req_plot_per_func(get_traces_one_video_all_users()[::], [
req_plot_per_func(get_traces_one_video_one_user()[::25], [
    req_tiles_voro24_fov_110radius_cover_center,
    req_tiles_voro14_fov_110radius_cover_center,
    req_tiles_voro24_fov_110x90_cover_any,
    req_tiles_voro14_fov_110x90_cover_any,
    req_tiles_rectan_fov_33perc_cover,
    req_tiles_rectan_fov_20perc_cover,
    req_tiles_rectan_fov_110radius_cover_center,
    req_tiles_voro14_fov_20perc_cover,
    req_tiles_voro14_fov_33perc_cover,
    req_tiles_voro24_fov_20perc_cover,
    req_tiles_voro24_fov_33perc_cover,
])
